# Magentic-One Creative Pipeline Orchestration

🎪 **Simple Multi-Agent Creative System**

This tutorial demonstrates:
- **Magentic-One orchestration** with Semantic Kernel
- **Three specialized Azure AI Agents** working together
- **Local plugins with feedback loops**
- **Creative content pipeline**: Joke → Haiku → French Translation

**Architecture:**
```
┌─────────────────────────────────────────────────────────────┐
│                    Magentic Orchestration                   │
│                (StandardMagenticManager)                    │
└─────────────────────┬───────────────────────────────────────┘
                      │
    ┌─────────────────┼─────────────────┐
    │                 │                 │
┌───▼────┐    ┌──────▼──────┐    ┌─────▼─────┐
│ Joke   │    │   Haiku     │    │  French   │
│Writer  │    │ Converter   │    │Translator │
│(Azure) │    │  (Azure)    │    │ (Azure)   │
└───┬────┘    └──────┬──────┘    └─────┬─────┘
    │                │                 │
┌───▼────┐    ┌──────▼──────┐    ┌─────▼─────┐
│Topic   │    │ Haiku       │    │    No     │
│Plugin  │    │Classifier   │    │  Plugin   │
│(Local) │    │ (Local)     │    │           │
└────────┘    └─────────────┘    └───────────┘
```

**Use Case**: "*Create a joke → convert to haiku → translate to French with feedback loops*"

---

## 🔧 Setup and Prerequisites

**Environment Variables Required:**
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name
- `AZURE_OPENAI_API_KEY`: Your Azure OpenAI API key
- `AZURE_OPENAI_ENDPOINT`: Your Azure OpenAI endpoint

**Azure Configuration:**
- Uses **Azure AI Agents** (not ChatCompletionAgent)
- Uses **AzureChatCompletion** service (not OpenAIChatCompletion)
- Automatically configures Azure OpenAI connection using environment variables
- Compatible with Azure AI Studio deployments

**Simple Architecture:**
1. **Azure AI Agents**: Deployed on Azure (managed service)
2. **Magentic Orchestrator**: Runs in local environment
3. **Local Plugins**: Topic selection, haiku classification
4. **No External APIs**: Simple, reliable testing scenario

**Creative Pipeline Flow:**
- Joke Writer selects topic and creates joke
- Haiku Converter transforms joke to haiku and classifies length
- French Translator converts approved haiku to French

In [1]:
# Install required packages for Magentic orchestration
# !pip install azure-ai-agents azure-identity semantic-kernel

import os
import json
import asyncio
import random
import tempfile
from datetime import datetime, timedelta
from typing import List

# Azure AI Agents and Semantic Kernel imports
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import (
    Agent,
    AzureAIAgent,
    MagenticOrchestration,
    StandardMagenticManager,
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent, AuthorRole
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

print("✅ Packages imported successfully!")
print(f"📅 Tutorial started at: {datetime.now()}")
print("🎪 Using Azure AI Agents for Creative Pipeline")

✅ Packages imported successfully!
📅 Tutorial started at: 2025-06-10 15:13:31.539057
🎪 Using Azure AI Agents for Creative Pipeline


## 🎯 Step 1: Test Environment Setup

Let's verify our environment is ready for the creative pipeline with no external dependencies.

In [2]:
# Simple environment verification
print("🎪 Testing Creative Pipeline Environment...")

# Check required environment variables
required_vars = ["PROJECT_ENDPOINT", "MODEL_DEPLOYMENT_NAME"]
missing_vars = []

for var in required_vars:
    if var in os.environ:
        print(f"✅ {var}: {'*' * (len(os.environ[var]) - 10)}{os.environ[var][-10:]}")
    else:
        missing_vars.append(var)
        print(f"❌ {var}: Missing")

if missing_vars:
    print(f"\n⚠️ Missing environment variables: {', '.join(missing_vars)}")
    print("Please set these before running the orchestration.")
else:
    print("\n✅ All required environment variables are set!")

# Test temporary file creation (used by haiku classifier)
try:
    with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=True) as temp_file:
        temp_file.write("Test haiku content")
        print("✅ Temporary file creation works!")
except Exception as e:
    print(f"❌ Temporary file creation failed: {e}")

print(f"\n🎯 Environment ready for creative pipeline at: {datetime.now()}")

🎪 Testing Creative Pipeline Environment...
✅ PROJECT_ENDPOINT: ***********************************************************************************ni-agentic
✅ MODEL_DEPLOYMENT_NAME: gpt-4.1

✅ All required environment variables are set!
✅ Temporary file creation works!

🎯 Environment ready for creative pipeline at: 2025-06-10 15:13:31.557218


## 🧩 Step 2: Create Creative Plugins

These plugins will run locally and provide the creative capabilities for our joke-haiku-translation pipeline.

In [3]:
class JokeTopicPlugin:
    """Plugin for joke generation with random topic selection"""
    
    def __init__(self):
        self.topics = [
            "cats", "programming", "coffee", "weather", "cooking",
            "travel", "sports", "technology", "books", "music"
        ]

    @kernel_function(
        description="Select a random topic for joke generation from a predefined list",
        name="get_random_topic"
    )
    def get_random_topic(self) -> str:
        """Returns a random topic for joke generation"""
        topic = random.choice(self.topics)
        print(f"🎯 Selected random topic: {topic}")
        return topic

    @kernel_function(
        description="Get all available joke topics",
        name="get_all_topics"
    )
    def get_all_topics(self) -> str:
        """Returns all available joke topics"""
        return json.dumps({"topics": self.topics}, indent=2)

# Initialize the plugin
joke_topic_plugin = JokeTopicPlugin()
print("✅ Joke Topic Plugin created successfully!")
print(f"📝 Available topics: {', '.join(joke_topic_plugin.topics)}")

✅ Joke Topic Plugin created successfully!
📝 Available topics: cats, programming, coffee, weather, cooking, travel, sports, technology, books, music


In [4]:
class HaikuClassifierPlugin:
    """Plugin for haiku analysis and classification"""
    
    @kernel_function(
        description="Classify haiku as short or long based on character count and save to temp file",
        name="classify_haiku_length"
    )
    def classify_haiku_length(self, haiku_text: str) -> str:
        """Classify haiku length and save to temporary file"""
        
        # Save haiku to temporary file
        with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False) as temp_file:
            temp_file.write(haiku_text)
            temp_filepath = temp_file.name
        
        # Count characters (excluding whitespace)
        char_count = len(haiku_text.replace(' ', '').replace('\n', ''))
        
        # Classification logic
        if char_count < 50:
            classification = "short"
            feedback = "Please generate a longer joke - the haiku is too short"
        elif char_count > 100:
            classification = "long" 
            feedback = "Please generate a shorter joke - the haiku is too long"
        else:
            classification = "perfect"
            feedback = "Good length - ready to send to translator"
        
        result = {
            "haiku_text": haiku_text,
            "character_count": char_count,
            "classification": classification,
            "feedback": feedback,
            "temp_file_path": temp_filepath,
            "ready_for_translation": classification == "perfect"
        }
        
        print(f"📏 Haiku classified: {classification} ({char_count} chars)")
        print(f"💾 Saved to: {temp_filepath}")
        
        return json.dumps(result, indent=2)

    @kernel_function(
        description="Read haiku from temporary file",
        name="read_haiku_from_file"
    )
    def read_haiku_from_file(self, file_path: str) -> str:
        """Read haiku content from specified file path"""
        try:
            with open(file_path, 'r') as file:
                content = file.read()
            return content
        except Exception as e:
            return f"Error reading file: {str(e)}"

# Initialize the plugin
haiku_classifier_plugin = HaikuClassifierPlugin()
print("✅ Haiku Classifier Plugin created successfully!")
print("📊 Classification rules: <50 chars = short, >100 chars = long, 50-100 chars = perfect")

✅ Haiku Classifier Plugin created successfully!
📊 Classification rules: <50 chars = short, >100 chars = long, 50-100 chars = perfect


## 🧪 Step 3: Test Creative Plugins

Let's test our local plugins to ensure they work correctly before creating the agents.

In [5]:
# Test the creative plugins
print("🧪 Testing Creative Plugins")
print("=" * 50)

# Test 1: Joke Topic Plugin
print("\n1. Testing Joke Topic Selection:")
random_topic = joke_topic_plugin.get_random_topic()
print(f"   Random topic: {random_topic}")

all_topics = joke_topic_plugin.get_all_topics()
print(f"   All topics: {json.loads(all_topics)['topics']}")

# Test 2: Haiku Classifier Plugin
print("\n2. Testing Haiku Classification:")

# Test with short haiku
short_haiku = "Cat sits\nOn warm laptop keys\nZzz..."
short_result = haiku_classifier_plugin.classify_haiku_length(short_haiku)
short_classification = json.loads(short_result)
print(f"   Short haiku result: {short_classification['classification']} ({short_classification['character_count']} chars)")

# Test with long haiku
long_haiku = """The programming bug hides deep within nested loops and complex algorithms
Coffee grows cold as developers search through infinite lines of code seeking solutions
Eureka moment arrives with dawn breaking through tired eyes and weary minds finally at rest"""
long_result = haiku_classifier_plugin.classify_haiku_length(long_haiku)
long_classification = json.loads(long_result)
print(f"   Long haiku result: {long_classification['classification']} ({long_classification['character_count']} chars)")

# Test with perfect haiku
perfect_haiku = """Morning coffee brews
Steam rises from ceramic cup
Productivity starts"""
perfect_result = haiku_classifier_plugin.classify_haiku_length(perfect_haiku)
perfect_classification = json.loads(perfect_result)
print(f"   Perfect haiku result: {perfect_classification['classification']} ({perfect_classification['character_count']} chars)")

# Test file reading
print(f"\n3. Testing File Reading:")
if perfect_classification.get('temp_file_path'):
    file_content = haiku_classifier_plugin.read_haiku_from_file(perfect_classification['temp_file_path'])
    print(f"   File content matches: {file_content == perfect_haiku}")

print("\n✅ All creative plugins tested successfully!")

🧪 Testing Creative Plugins

1. Testing Joke Topic Selection:
🎯 Selected random topic: music
   Random topic: music
   All topics: ['cats', 'programming', 'coffee', 'weather', 'cooking', 'travel', 'sports', 'technology', 'books', 'music']

2. Testing Haiku Classification:
📏 Haiku classified: short (29 chars)
💾 Saved to: C:\Users\SELHOU~1\AppData\Local\Temp\tmpcx_vxgee.txt
   Short haiku result: short (29 chars)
📏 Haiku classified: long (215 chars)
💾 Saved to: C:\Users\SELHOU~1\AppData\Local\Temp\tmpvalrsgc2.txt
   Long haiku result: long (215 chars)
📏 Haiku classified: perfect (60 chars)
💾 Saved to: C:\Users\SELHOU~1\AppData\Local\Temp\tmpj2mma32b.txt
   Perfect haiku result: perfect (60 chars)

3. Testing File Reading:
   File content matches: True

✅ All creative plugins tested successfully!


## 🤖 Step 4: Create Specialized Creative Agents

We'll create three specialized agents that will work together in the Magentic orchestration:
1. **Joke Writer**: Selects topics and creates jokes
2. **Haiku Converter**: Transforms jokes to haikus and provides feedback
3. **French Translator**: Translates approved haikus to French

In [ ]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print agent messages"""
    if message.role != AuthorRole.TOOL:
        print(f"\n🎭 **{message.name}**")
        print(f"   {message.content}")
        print("-" * 60)

async def create_creative_agents() -> List[Agent]:
    """Create three specialized agents for the joke-haiku-translation pipeline"""
    
    endpoint = os.environ["PROJECT_ENDPOINT"]
    model_deployment = os.environ["MODEL_DEPLOYMENT_NAME"]
    agents = []
    
    print("🎪 Creating Creative Pipeline Agents...")
    
    async with DefaultAzureCredential() as creds:
        client = AzureAIAgent.create_client(credential=creds, endpoint=endpoint)
        
        # 1. Joke Writer Agent
        print("😂 Creating Joke Writer Agent...")
        joke_writer_definition = await client.agents.create_agent(
            model=model_deployment,
            name="joke_writer",
            description="Creative joke writer with topic selection capabilities",
            instructions="""
            You are a creative joke writer. Your role is to:
            - Use the available plugin to select random topics for jokes
            - Write funny, clean jokes based on the selected topics
            - Adjust joke length based on feedback from the haiku agent
            - If told to make jokes longer, add more setup or details
            - If told to make jokes shorter, be more concise and punchy
            
            Always be creative and family-friendly. Listen to feedback and adjust accordingly.
            """
        )
        
        joke_writer = AzureAIAgent(
            client=client,
            definition=joke_writer_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000)),
            plugins=[joke_topic_plugin]
        )
        agents.append(joke_writer)
        print(f"✅ Joke Writer: {joke_writer_definition.id}")
        
        # 2. Haiku Converter Agent  
        print("🌸 Creating Haiku Converter Agent...")
        haiku_converter_definition = await client.agents.create_agent(
            model=model_deployment,
            name="haiku_converter",
            description="Converts jokes to haikus and provides length feedback",
            instructions="""
            You are a haiku poetry expert. Your role is to:
            - Convert jokes into traditional haiku format (5-7-5 syllable pattern)
            - Use the classifier plugin to analyze haiku length
            - Provide feedback to the joke writer about length adjustments needed
            - Only approve haikus that are the right length for translation
            
            Remember: Traditional haiku should capture the essence of the joke while following the 5-7-5 syllable structure.
            Use your plugin to classify the length and provide appropriate feedback.
            """
        )
        
        haiku_converter = AzureAIAgent(
            client=client,
            definition=haiku_converter_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000)),
            plugins=[haiku_classifier_plugin]
        )
        agents.append(haiku_converter)
        print(f"✅ Haiku Converter: {haiku_converter_definition.id}")
        
        # 3. French Translator Agent
        print("🇫🇷 Creating French Translator Agent...")
        french_translator_definition = await client.agents.create_agent(
            model=model_deployment,
            name="french_translator",
            description="Translates haikus from English to French",
            instructions="""
            You are an expert French translator specializing in poetry. Your role is to:
            - Translate English haikus into beautiful, flowing French
            - Maintain the poetic essence and rhythm when possible
            - Ensure the translation captures the humor of the original joke
            - Provide both the French translation and a brief explanation of any cultural adaptations made
            
            Focus on creating elegant French that preserves both the meaning and artistic quality of the haiku.
            Just output the translated haiku with no additional commentary or explanations.
            """
        )
        
        french_translator = AzureAIAgent(
            client=client,
            definition=french_translator_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000)),
            # No plugins - pure translation work
        )
        agents.append(french_translator)
        print(f"✅ French Translator: {french_translator_definition.id}")
    
    print(f"✅ Created {len(agents)} agents total")
    return agents

# Test agent creation
print("🎪 Ready to create creative agents...")

🎪 Ready to create creative agents...


## 🎭 Step 5: Create Magentic-One Creative Orchestration

Now we'll create the Magentic orchestration that will coordinate our creative agents to transform jokes into French haikus.

In [7]:
async def create_creative_orchestration() -> MagenticOrchestration:
    """Create and configure the creative Magentic orchestration"""
    
    print("🎭 Setting up Creative Magentic-One orchestration...")
    
    # Create creative agents
    agents = await create_creative_agents()
    
    # Configure Azure Chat Completion for the manager
    manager_azure_chat_completion = AzureChatCompletion()
    
    # Create the Magentic orchestration
    # The StandardMagenticManager coordinates agent interactions
    creative_orchestration = MagenticOrchestration(
        members=agents,
        manager=StandardMagenticManager(
            chat_completion_service=manager_azure_chat_completion
        ),
        agent_response_callback=agent_response_callback,
    )
    
    print(f"✅ Creative orchestration created with {len(agents)} agents")
    print("   Agents: " + ", ".join([agent.name for agent in agents]))
    
    return creative_orchestration

print("🎯 Creative orchestration configuration ready")

🎯 Creative orchestration configuration ready


## 🚀 Step 6: Execute Creative Pipeline Task

Now let's execute our creative use case: **"Create a joke → convert to haiku → translate to French"**

This will involve:
1. Joke Writer selecting a topic and creating a joke
2. Haiku Converter transforming the joke and checking length
3. French Translator converting the approved haiku
4. Feedback loops for length adjustments

In [8]:
async def execute_creative_pipeline_task():
    """Execute the creative pipeline task using Magentic orchestration"""
    
    print("🎪 Starting Creative Pipeline Task")
    print("=" * 70)
    
    # Create the orchestration
    orchestration = await create_creative_orchestration()
    
    # Create and start runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        # Define the creative task
        creative_task = """
        I want you to create a creative pipeline:
        
        1. Joke Writer: Select a random topic and write a funny joke about it
        2. Haiku Converter: Convert the joke into a haiku and check if the length is appropriate
           - If too short, ask joke writer for a longer joke
           - If too long, ask joke writer for a shorter joke  
           - If just right, proceed to translation
        3. French Translator: Translate the approved haiku into beautiful French
        
        Please work together to create one perfect joke-haiku-translation sequence.
        Make sure to use the plugins for topic selection and haiku classification.
        """
        
        print("📋 Creative Task Definition:")
        print(creative_task)
        print("\n" + "=" * 70)
        print("🎭 Beginning Creative Orchestration...")
        print("=" * 70)
        
        # Execute the orchestration
        orchestration_result = await orchestration.invoke(
            task=creative_task,
            runtime=runtime
        )
        
        # Wait for the results
        result = await orchestration_result.get()

        # Display final results
        print("\n" + "=" * 70)
        print("🎯 CREATIVE ORCHESTRATION COMPLETE")
        print("\n🎨 Final Creative Result:")
        print(f"\n{result}")
        print("=" * 70)
        
    except Exception as e:
        print(f"❌ Error during orchestration: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        # Cleanup
        await runtime.stop()
        print("\n🧹 Runtime stopped and resources cleaned up")

# Execute the creative task
print("🌟 Ready to execute creative pipeline task!")
print("This will involve three agents working together through Magentic orchestration.")

🌟 Ready to execute creative pipeline task!
This will involve three agents working together through Magentic orchestration.


In [9]:
# Run the creative pipeline task
await execute_creative_pipeline_task()

🎪 Starting Creative Pipeline Task
🎭 Setting up Creative Magentic-One orchestration...
🎪 Creating Creative Pipeline Agents...
😂 Creating Joke Writer Agent...
✅ Joke Writer: asst_YkKc5qE7gmJeWDJWZqBhb7dA
🌸 Creating Haiku Converter Agent...
✅ Haiku Converter: asst_v8uBqXe9jJfvncULUyTTT3sA
🇫🇷 Creating French Translator Agent...
✅ French Translator: asst_ijXBpExgMVsDKKK2JiYHmbli
✅ Created 3 agents total
✅ Creative orchestration created with 3 agents
   Agents: joke_writer, haiku_converter, french_translator
📋 Creative Task Definition:

        I want you to create a creative pipeline:

        1. Joke Writer: Select a random topic and write a funny joke about it
        2. Haiku Converter: Convert the joke into a haiku and check if the length is appropriate
           - If too short, ask joke writer for a longer joke
           - If too long, ask joke writer for a shorter joke  
           - If just right, proceed to translation
        3. French Translator: Translate the approved haiku int

## 🧪 Step 7: Test Individual Creative Agents

Let's test our creative agents individually to ensure they work correctly.

In [10]:
async def test_individual_creative_agents():
    """Test individual creative agents manually"""
    
    print("🧪 Testing Individual Creative Agents")
    print("=" * 50)
    
    endpoint = os.environ["PROJECT_ENDPOINT"]
    model_deployment = os.environ["MODEL_DEPLOYMENT_NAME"]
    
    # Create individual agents for testing
    async with DefaultAzureCredential() as creds:
        client = AzureAIAgent.create_client(credential=creds, endpoint=endpoint)
        
        # Test Joke Writer Agent
        print("\n1. Testing Joke Writer Agent:")
        joke_agent_definition = await client.agents.create_agent(
            model=model_deployment,
            name="test_joke_writer",
            instructions="You are a joke writer. Use your plugin to select a topic and write a short joke about it."
        )
        
        joke_agent = AzureAIAgent(
            client=client,
            definition=joke_agent_definition,
            plugins=[joke_topic_plugin]
        )
        
        test_joke_query = "Please select a random topic and write a short joke about it."
        print(f"   Query: {test_joke_query}")
        
        async for response in joke_agent.invoke(test_joke_query):
            if response.role != AuthorRole.TOOL:
                print(f"   Response: {response.content}")
                break
        
        # Test Haiku Converter Agent
        print("\n2. Testing Haiku Converter Agent:")
        haiku_agent_definition = await client.agents.create_agent(
            model=model_deployment,
            name="test_haiku_converter",
            instructions="You convert text to haikus and classify their length using your plugin."
        )
        
        haiku_agent = AzureAIAgent(
            client=client,
            definition=haiku_agent_definition,
            plugins=[haiku_classifier_plugin]
        )
        
        test_haiku_query = "Convert this joke to a haiku and classify its length: Why do programmers prefer dark mode? Because light attracts bugs!"
        print(f"   Query: {test_haiku_query}")
        
        async for response in haiku_agent.invoke(test_haiku_query):
            if response.role != AuthorRole.TOOL:
                print(f"   Response: {response.content}")
                break

print("🧪 Individual agent testing ready")

🧪 Individual agent testing ready


In [11]:
# Run individual agent tests
await test_individual_creative_agents()

🧪 Testing Individual Creative Agents

1. Testing Joke Writer Agent:
   Query: Please select a random topic and write a short joke about it.
🎯 Selected random topic: coffee
   Response: Topic: Coffee

Joke: Why did the coffee file a police report? It got mugged!

2. Testing Haiku Converter Agent:
   Query: Convert this joke to a haiku and classify its length: Why do programmers prefer dark mode? Because light attracts bugs!
📏 Haiku classified: perfect (62 chars)
💾 Saved to: C:\Users\SELHOU~1\AppData\Local\Temp\tmpz78mr5nt.txt
📏 Haiku classified: perfect (62 chars)
💾 Saved to: C:\Users\SELHOU~1\AppData\Local\Temp\tmpz9x6jgk1.txt
   Response: Here are two haiku versions of your joke, both classified as "perfect" length:

1.
Programmers at night  
Dark mode keeps bugs far away  
Light draws them closer

2.
Why dark mode, you ask?  
Because light brings all the bugs  
Code safer in dark

Both are strong haikus and ready to use!


## 🎯 Step 8: Summary and Cleanup

Let's clean up resources and provide a summary of what we've accomplished.

In [12]:
# Cleanup and summary
print("🧹 Cleaning up resources...")

print("✅ Cleanup completed")

print("\n" + "=" * 70)
print("🎪 CREATIVE PIPELINE TUTORIAL SUMMARY")
print("=" * 70)

summary = """
🌟 What We Accomplished:

✅ Simple Architecture Implementation:
   • Three Azure AI Agents working together
   • Local plugins for creative capabilities
   • Magentic-One orchestration coordination
   • No external API dependencies

✅ Agent Specialization:
   • Joke Writer: Topic selection and joke creation
   • Haiku Converter: Poetry transformation and length analysis
   • French Translator: Language translation

✅ Technical Components:
   • Local Python plugins with @kernel_function decorators
   • File-based temporary storage for haiku classification
   • Random topic selection from predefined list
   • Feedback loops between agents

✅ Creative Task Execution:
   • "Create joke → convert to haiku → translate to French"
   • Multi-agent coordination through Magentic orchestration
   • Length-based feedback for joke adjustment
   • Cultural adaptation in French translation

🏗️ System Architecture Benefits:
   • Simple: No external dependencies or complex APIs
   • Reliable: Local plugins reduce network failure points
   • Educational: Clear demonstration of agent collaboration
   • Testable: Easy to debug and understand

💡 Key Learning Points:
   • Magentic orchestration enables complex multi-agent workflows
   • Local plugins provide specialized capabilities
   • Feedback loops allow iterative improvement
   • Azure AI Agents can be combined with local processing
   • Creative use cases are perfect for demonstrating agent coordination
"""

print(summary)

print("\n🎉 Creative Pipeline Tutorial completed successfully!")
print(f"⏰ Completed at: {datetime.now()}")

🧹 Cleaning up resources...
✅ Cleanup completed

🎪 CREATIVE PIPELINE TUTORIAL SUMMARY

🌟 What We Accomplished:

✅ Simple Architecture Implementation:
   • Three Azure AI Agents working together
   • Local plugins for creative capabilities
   • Magentic-One orchestration coordination
   • No external API dependencies

✅ Agent Specialization:
   • Joke Writer: Topic selection and joke creation
   • Haiku Converter: Poetry transformation and length analysis
   • French Translator: Language translation

✅ Technical Components:
   • Local Python plugins with @kernel_function decorators
   • File-based temporary storage for haiku classification
   • Random topic selection from predefined list
   • Feedback loops between agents

✅ Creative Task Execution:
   • "Create joke → convert to haiku → translate to French"
   • Multi-agent coordination through Magentic orchestration
   • Length-based feedback for joke adjustment
   • Cultural adaptation in French translation

🏗️ System Architecture Ben